In [1]:
import functools
import itertools
import logging
import math
import os
import pickle
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import seaborn as sns
import yaml
from itertools import product

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 12.0)})
sns.set_style("whitegrid")

import numpy as np
import pandas as pd
import torch.nn.functional as F

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from lda4rec.datasets import Interactions, DataLoader, random_train_test_split
from lda4rec.evaluations import mrr_score, precision_recall_score, summary
from lda4rec.estimators import MFEst, PopEst, LDA4RecEst, SNMFEst
from lda4rec.utils import process_ids, cmp_ranks

In [3]:
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import torch
from pyro.distributions import constraints
from pyro.infer import SVI, Predictive, Trace_ELBO, TraceEnum_ELBO, config_enumerate

In [4]:
import neptune.new as neptune
# init dummy neptune to avoid problems with logging
neptune.init(mode='offline');

offline/eeec9147-d217-455b-890a-d996887ea529


## Results exported from Neptune.AI

In [5]:
df = pd.read_csv("../results/experiments.csv")

In [6]:
df.rename(columns=lambda x: x.split("/")[-1].split(" ")[0], inplace=True)

In [7]:
metrics = {f"{m}_{s}": np.mean for m, s in product(["mrr", "prec", "recall"], ["train", "valid", "test"])}
idx_order = ["PopEst", "NMFEst", "SNMFEst", "MFEst", "LDA4RecEst"]

In [8]:
df.loc[(df["dataset"] == "movielens-1m")].groupby(["estimator", "embedding_dim"], dropna=False).agg(metrics).reindex(index=idx_order, level=0)

mrr_train  mrr_valid  mrr_test  prec_train  \
estimator embedding_dim                                               
PopEst    NaN             0.023706   0.032858  0.033488    0.339636   
NMFEst    4.0             0.022534   0.031129  0.031757    0.338088   
          8.0             0.023526   0.032614  0.033156    0.339216   
          16.0            0.023637   0.032580  0.033320    0.339205   
          32.0            0.023615   0.032365  0.033103    0.338433   
          48.0            0.023515   0.032043  0.032790    0.335996   
          64.0            0.023238   0.031562  0.032424    0.336004   
SNMFEst   4.0             0.029322   0.043518  0.044251    0.412130   
          8.0             0.032732   0.051700  0.051630    0.460662   
          16.0            0.037630   0.063475  0.062563    0.519407   
          32.0            0.040289   0.069753  0.067168    0.548780   
          48.0            0.041993   0.071463  0.069270    0.565281   
          64.0            0.043409   0.072510  0.070517    0.578996   
MFEst     4.0             0.030659   0.047478  0.048307    0.436071   
          8.0             0.035171   0.058181  0.058181    0.492817   
          16.0            0.038462   0.064867  0.063977    0.528877   
          32.0            0.041237   0.068536  0.066223    0.553598   
          48.0            0.043075   0.069245  0.066615    0.571600   
          64.0            0.045014   0.068006  0.067792    0.588811   

                         prec_valid  prec_test  recall_train  recall_valid  \
estimator embedding_dim                                                      
PopEst    NaN              0.032154   0.033084      0.045557      0.064532   
NMFEst    4.0              0.032115   0.033138      0.045410      0.064410   
          8.0              0.032139   0.033176      0.045510      0.064685   
          16.0             0.032192   0.033259      0.045494      0.064726   
          32.0             0.031941   0.032782      0.045527      0.064019   
          48.0             0.031903   0.032737      0.045327      0.064162   
          64.0             0.031868   0.032708      0.045241      0.063726   
SNMFEst   4.0              0.043205   0.043614      0.057985      0.089223   
          8.0              0.051649   0.051628      0.066209      0.108178   
          16.0             0.062570   0.061028      0.078043      0.133402   
          32.0             0.067481   0.065552      0.084599      0.146745   
          48.0             0.069356   0.067550      0.088438      0.152300   
          64.0             0.070135   0.068697      0.091434      0.153446   
MFEst     4.0              0.047080   0.046846      0.061627      0.096663   
          8.0              0.057111   0.055825      0.072491      0.121395   
          16.0             0.064393   0.062587      0.080149      0.138847   
          32.0             0.066649   0.065004      0.086069      0.145797   
          48.0             0.066941   0.065060      0.090133      0.147518   
          64.0             0.066044   0.065241      0.094338      0.145731   

                         recall_test  
estimator embedding_dim               
PopEst    NaN               0.065908  
NMFEst    4.0               0.066450  
          8.0               0.066027  
          16.0              0.066263  
          32.0              0.065677  
          48.0              0.065991  
          64.0              0.065825  
SNMFEst   4.0               0.090498  
          8.0               0.107532  
          16.0              0.129522  
          32.0              0.141108  
          48.0              0.145514  
          64.0              0.150908  
MFEst     4.0               0.097700  
          8.0               0.118196  
          16.0              0.133671  
          32.0              0.139404  
          48.0              0.140946  
          64.0              0.144292

In [9]:
df.loc[(df["dataset"] == "goodbooks")].groupby(["estimator", "embedding_dim"], dropna=False).agg(metrics).reindex(index=idx_order, level=0)

mrr_train  mrr_valid  mrr_test  prec_train  \
estimator embedding_dim                                               
PopEst    NaN             0.015454   0.023697  0.023918    0.297855   
NMFEst    4.0             0.009325   0.013302  0.013347    0.234443   
          8.0             0.008760   0.012447  0.012470    0.229933   
          16.0            0.012145   0.018133  0.018254    0.276424   
          32.0            0.017977   0.028189  0.028376    0.304750   
          48.0            0.019474   0.031127  0.031395    0.301031   
          64.0            0.020352   0.032814  0.033168    0.306039   
SNMFEst   4.0             0.020552   0.035483  0.035509    0.401228   
          8.0             0.023429   0.041675  0.042030    0.448016   
          16.0            0.026163   0.048722  0.048844    0.495876   
          32.0            0.028218   0.054817  0.054590    0.531036   
          48.0            0.029462   0.058042  0.058170    0.551713   
          64.0            0.030486   0.060685  0.060743    0.568836   
MFEst     4.0             0.021690   0.037178  0.037361    0.421354   
          8.0             0.024290   0.044041  0.044052    0.463748   
          16.0            0.026592   0.050465  0.050721    0.504217   
          32.0            0.028436   0.055303  0.055351    0.535559   
          48.0            0.029558   0.057983  0.057920    0.553680   
          64.0            0.030378   0.059358  0.059449    0.566875   

                         prec_valid  prec_test  recall_train  recall_valid  \
estimator embedding_dim                                                      
PopEst    NaN              0.026989   0.027079      0.029935      0.047702   
NMFEst    4.0              0.019982   0.019691      0.023328      0.034696   
          8.0              0.018830   0.018680      0.022863      0.032696   
          16.0             0.024649   0.024602      0.027715      0.043186   
          32.0             0.027921   0.027900      0.030696      0.049326   
          48.0             0.031307   0.031371      0.030352      0.055490   
          64.0             0.034758   0.035001      0.030945      0.061762   
SNMFEst   4.0              0.039865   0.039857      0.040393      0.070296   
          8.0              0.046693   0.046743      0.045997      0.083610   
          16.0             0.054867   0.054872      0.051690      0.099578   
          32.0             0.061892   0.061933      0.055600      0.112821   
          48.0             0.066105   0.066236      0.057862      0.120812   
          64.0             0.069477   0.069582      0.059717      0.127072   
MFEst     4.0              0.042074   0.042265      0.042876      0.074786   
          8.0              0.048954   0.049005      0.048062      0.088400   
          16.0             0.056346   0.056614      0.052566      0.102128   
          32.0             0.062390   0.062411      0.056040      0.113742   
          48.0             0.065851   0.065852      0.058073      0.120358   
          64.0             0.068167   0.067976      0.059538      0.124743   

                         recall_test  
estimator embedding_dim               
PopEst    NaN               0.047867  
NMFEst    4.0               0.034357  
          8.0               0.032619  
          16.0              0.043182  
          32.0              0.049352  
          48.0              0.055785  
          64.0              0.062428  
SNMFEst   4.0               0.070744  
          8.0               0.084337  
          16.0              0.099674  
          32.0              0.112936  
          48.0              0.121081  
          64.0              0.127380  
MFEst     4.0               0.075502  
          8.0               0.088667  
          16.0              0.103009  
          32.0              0.113782  
          48.0              0.120510  
          64.0              0.124612

In [10]:
df.loc[df["dataset"] == "movielens-100k"].groupby(["estimator", "embedding_dim",], dropna=False).agg(metrics).reindex(index=idx_order, level=0)

mrr_train  mrr_valid  mrr_test  prec_train  \
estimator  embedding_dim                                               
PopEst     NaN             0.026137   0.042217  0.037069    0.328525   
MFEst      2.0             0.026419   0.041727  0.041071    0.333115   
           4.0             0.030171   0.054383  0.049304    0.370984   
           8.0             0.031930   0.054979  0.047378    0.387869   
           16.0            0.035087   0.056758  0.046867    0.436557   
           32.0            0.039163   0.054841  0.048462    0.500492   
LDA4RecEst 2.0             0.031015   0.047613  0.052898    0.385902   
           4.0             0.034830   0.056307  0.066236    0.426393   
           8.0             0.034740   0.043342  0.053524    0.419180   
           16.0            0.038174   0.050153  0.058407    0.464426   
           32.0            0.039091   0.051754  0.058738    0.493115   

                          prec_valid  prec_test  recall_train  recall_valid  \
estimator  embedding_dim                                                      
PopEst     NaN              0.026799   0.031148      0.054219      0.069386   
MFEst      2.0              0.029137   0.033151      0.054949      0.077431   
           4.0              0.040108   0.041530      0.063030      0.107724   
           8.0              0.040468   0.042987      0.067500      0.113402   
           16.0             0.042446   0.045173      0.072697      0.115023   
           32.0             0.043165   0.049909      0.081072      0.106547   
LDA4RecEst 2.0              0.041007   0.040984      0.066740      0.108132   
           4.0              0.045863   0.048816      0.076604      0.133678   
           8.0              0.046403   0.045902      0.072968      0.115465   
           16.0             0.044245   0.046995      0.082597      0.106134   
           32.0             0.043525   0.044991      0.084568      0.113190   

                          recall_test  
estimator  embedding_dim               
PopEst     NaN               0.072635  
MFEst      2.0               0.075098  
           4.0               0.099996  
           8.0               0.110723  
           16.0              0.111719  
           32.0              0.116455  
LDA4RecEst 2.0               0.105045  
           4.0               0.130728  
           8.0               0.119682  
           16.0              0.125783  
           32.0              0.115292

## Extract test scores by selecting the best validation score

In [11]:
def get_best_valid(df):
    """Select test score by best validation score"""
    results = {}
    for metric in ["mrr", "prec", "recall"]:
        best = df[f"{metric}_valid"].max()
        results[metric] = df[f"{metric}_test"].loc[df[f"{metric}_valid"] == best].values[0]
    return pd.Series(results)

In [12]:
mf_cmp_df = (
    df.loc[df["dataset"].isin(["movielens-1m", "goodbooks"])]
     .groupby(["embedding_dim", "estimator", "dataset"], dropna=False)
     .apply(get_best_valid)
     .reset_index()
     .melt(id_vars=["embedding_dim", "estimator", "dataset"], value_vars=["mrr", "prec", "recall"], var_name="metric")
     .sort_values(["embedding_dim", "estimator", "dataset"])
     .pivot(index=["embedding_dim", "estimator"], values="value", columns=["dataset", "metric"])
     .reindex(index=idx_order, level=1)
)
mf_cmp_df

dataset                 goodbooks                     movielens-1m            \
metric                        mrr      prec    recall          mrr      prec   
embedding_dim estimator                                                        
NaN           PopEst     0.023918  0.027079  0.047867     0.033488  0.033084   
4.0           NMFEst     0.014388  0.022056  0.038420     0.032927  0.033031   
              SNMFEst    0.036186  0.040912  0.072584     0.046642  0.046211   
              MFEst      0.038901  0.044045  0.079124     0.050495  0.048702   
8.0           NMFEst     0.015121  0.019261  0.033310     0.033445  0.033191   
              SNMFEst    0.042435  0.047185  0.085326     0.053639  0.052028   
              MFEst      0.044683  0.049835  0.090115     0.058240  0.057044   
16.0          NMFEst     0.019945  0.026436  0.046623     0.033461  0.033351   
              SNMFEst    0.049671  0.055800  0.101652     0.062695  0.061526   
              MFEst      0.050875  0.057127  0.103747     0.063849  0.062131   
32.0          NMFEst     0.028766  0.028268  0.050012     0.033223  0.033155   
              SNMFEst    0.055048  0.062215  0.113179     0.068511  0.066453   
              MFEst      0.056080  0.062841  0.114629     0.064506  0.064888   
48.0          NMFEst     0.032190  0.033548  0.060065     0.032996  0.033084   
              SNMFEst    0.058595  0.066861  0.122292     0.068369  0.068143   
              MFEst      0.058730  0.066321  0.121440     0.067427  0.065777   
64.0          NMFEst     0.034171  0.039272  0.070492     0.032925  0.032978   
              SNMFEst    0.061561  0.070156  0.128119     0.069775  0.069050   
              MFEst      0.060261  0.068837  0.126254     0.067474  0.066489   

dataset                            
metric                     recall  
embedding_dim estimator            
NaN           PopEst     0.065908  
4.0           NMFEst     0.066281  
              SNMFEst    0.097219  
              MFEst      0.103090  
8.0           NMFEst     0.066516  
              SNMFEst    0.108542  
              MFEst      0.119924  
16.0          NMFEst     0.066191  
              SNMFEst    0.130733  
              MFEst      0.131973  
32.0          NMFEst     0.064652  
              SNMFEst    0.141667  
              MFEst      0.138600  
48.0          NMFEst     0.066660  
              SNMFEst    0.146653  
              MFEst      0.143905  
64.0          NMFEst     0.066924  
              SNMFEst    0.151497  
              MFEst      0.145744

In [13]:
print(mf_cmp_df.to_latex())

\begin{tabular}{llrrrrrr}
\toprule
     & dataset & \multicolumn{3}{l}{goodbooks} & \multicolumn{3}{l}{movielens-1m} \\
     & metric &       mrr &      prec &    recall &          mrr &      prec &    recall \\
embedding\_dim & estimator &           &           &           &              &           &           \\
\midrule
NaN & PopEst &  0.023918 &  0.027079 &  0.047867 &     0.033488 &  0.033084 &  0.065908 \\
4.0  & NMFEst &  0.014388 &  0.022056 &  0.038420 &     0.032927 &  0.033031 &  0.066281 \\
     & SNMFEst &  0.036186 &  0.040912 &  0.072584 &     0.046642 &  0.046211 &  0.097219 \\
     & MFEst &  0.038901 &  0.044045 &  0.079124 &     0.050495 &  0.048702 &  0.103090 \\
8.0  & NMFEst &  0.015121 &  0.019261 &  0.033310 &     0.033445 &  0.033191 &  0.066516 \\
     & SNMFEst &  0.042435 &  0.047185 &  0.085326 &     0.053639 &  0.052028 &  0.108542 \\
     & MFEst &  0.044683 &  0.049835 &  0.090115 &     0.058240 &  0.057044 &  0.119924 \\
16.0 & NMFEst &  0.019945 &  0.

In [14]:
lda4rec_cmp_df = (
    df.loc[(df["dataset"] == "movielens-100k")]
    .groupby(["estimator", "embedding_dim"], dropna=False)
    .apply(get_best_valid)
    .reindex(index=idx_order, level=0)
)
lda4rec_cmp_df

mrr      prec    recall
estimator  embedding_dim                              
PopEst     NaN            0.037069  0.031148  0.072635
MFEst      2.0            0.041071  0.033151  0.075098
           4.0            0.049304  0.041530  0.099996
           8.0            0.047378  0.042987  0.110723
           16.0           0.046867  0.045173  0.111719
           32.0           0.048462  0.049909  0.116455
LDA4RecEst 2.0            0.052898  0.040984  0.105045
           4.0            0.066236  0.048816  0.130728
           8.0            0.053524  0.045902  0.119682
           16.0           0.058407  0.046995  0.125783
           32.0           0.058738  0.044991  0.115292

In [15]:
print(lda4rec_cmp_df.to_latex())

\begin{tabular}{llrrr}
\toprule
           &      &       mrr &      prec &    recall \\
estimator & embedding\_dim &           &           &           \\
\midrule
PopEst & NaN  &  0.037069 &  0.031148 &  0.072635 \\
MFEst & 2.0  &  0.041071 &  0.033151 &  0.075098 \\
           & 4.0  &  0.049304 &  0.041530 &  0.099996 \\
           & 8.0  &  0.047378 &  0.042987 &  0.110723 \\
           & 16.0 &  0.046867 &  0.045173 &  0.111719 \\
           & 32.0 &  0.048462 &  0.049909 &  0.116455 \\
LDA4RecEst & 2.0  &  0.052898 &  0.040984 &  0.105045 \\
           & 4.0  &  0.066236 &  0.048816 &  0.130728 \\
           & 8.0  &  0.053524 &  0.045902 &  0.119682 \\
           & 16.0 &  0.058407 &  0.046995 &  0.125783 \\
           & 32.0 &  0.058738 &  0.044991 &  0.115292 \\
\bottomrule
\end{tabular}

